# Imports

In [ ]:
# Data Manipulation, Linear Algebra
import pandas as pd
import numpy as np

# Plots
import seaborn as sns
sns.set_style("darkgrid")
import matplotlib.pyplot as plt

#ignore warning messages 
import warnings
warnings.filterwarnings('ignore')

# Data Preprocessing

## Getting the Data

In [ ]:
base_path = "../input/tabular-playground-series-jan-2022/"

train_data = pd.read_csv(base_path + "train.csv")
test_data = pd.read_csv(base_path + "test.csv")
sample_submission_data = pd.read_csv(base_path + "sample_submission.csv")

In [ ]:
train_data.head()

In [ ]:
test_data.head()

## Concatenating Training and Testing Data

In [ ]:
full_data = pd.concat([train_data.iloc[:, :-1], test_data])
full_data.reset_index(drop=True, inplace=True)
full_data

## Feature Engineering

### Getting Year, Month and Day from Date

In [ ]:
full_data["year"] = full_data["date"].apply(lambda x: int(x[:4]))
full_data["month"] = full_data["date"].apply(lambda x: int(x[5:7]))
full_data["day"] = full_data["date"].apply(lambda x: int(x[8:]))

# Dropping the "date" and "row_id" Column
full_data.drop(columns=["date", "row_id"], inplace=True)

In [ ]:
full_data

# Analysis and EDA on FullData

In [ ]:
def Pie_Bar_Plot(data, col):
    # Gettting Column Value Counts
    column_value_counts = full_data[col].value_counts()

    # Creating the explode values_list
    explode_len = len(column_value_counts) - 1
    explode = [0]
    for i in range(explode_len):
        explode.append(0.1)

    # Creating Subplots
    f, ax = plt.subplots(1, 2, figsize=(18, 8))

    # Pie Plot
    column_value_counts.plot.pie(explode=explode, autopct='%1.1f%%', ax=ax[0], shadow=True)
    ax[0].set_title(f'{col.capitalize()} Pie Plot', fontsize=16, fontweight="bold")
    ax[0].set_ylabel('')

    # Barplot / Count Plot
    sns.barplot(x=column_value_counts.index, y=column_value_counts.values, ax=ax[1])
    ax[1].set_title(f'{col.capitalize()} Count Plot', fontsize=16, fontweight="bold")
    plt.show()

    print("\n\n")

## Distribution Plots using Pie and Bar Plot

In [ ]:
for col in full_data.columns:
    Pie_Bar_Plot(full_data, col)

## Crosstabs

In [ ]:
pd.crosstab(full_data.country, full_data.store, margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
pd.crosstab(full_data.country, full_data.year, margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
pd.crosstab(full_data.country, full_data.month, margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
pd.crosstab(full_data.year, full_data.month, margins=True).style.background_gradient(cmap='summer_r')

## Year vs Number of Sales (for Training Data)

In [ ]:
train_data["date"] = pd.to_datetime(train_data["date"])

Thanks to https://www.kaggle.com/vad13irt/tps-jan-2022-exploratory-data-analysis for plot idea below.

In [ ]:
def Date_Sales_Plot(data, hue=None):
    plt.figure(figsize=(25, 6))
    sns.lineplot(data=train_data, x="date", y="num_sold", hue=hue, err_style=None)
    plt.xlabel("Date", fontsize=12, fontweight="bold")
    plt.ylabel("Products Sold", fontsize=12, fontweight="bold")
    plt.title("Date vs Products Sold", fontsize=16, fontweight="bold")
    if hue:
        plt.legend(fontsize=13, edgecolor=None, facecolor=None, markerscale=2, handlelength=1, title=None)
    plt.show()

In [ ]:
Date_Sales_Plot(train_data)

In [ ]:
Date_Sales_Plot(train_data, "country")

In [ ]:
Date_Sales_Plot(train_data, "store")

In [ ]:
Date_Sales_Plot(train_data, "product")

# Preparing Data

## OneHotEncoding Categorical Variables

In [ ]:
full_data = pd.concat([pd.get_dummies(full_data[["country", "store", "product"]]), full_data[["year", "month", "day"]]], axis=1)

In [ ]:
# Getting only last two digits of the year
full_data["year"] = full_data["year"].apply(lambda x: int(str(x)[2:]))
full_data

In [ ]:
# Seperating Training and Testing Data
train = full_data.iloc[:len(train_data), :]
test = full_data.iloc[len(train_data):, :]

# Machine Learning Model

In [ ]:
# Train Test Split for training and testing the Regressor
from sklearn.model_selection import train_test_split

# Different Mathematical functions to calculate the Accuracy of Regression Model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Using CatBoostRegressor
from catboost import CatBoostRegressor

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, train_data["num_sold"], test_size = 0.2, random_state=0)

In [ ]:
# Initializing the Regressor
regressor = CatBoostRegressor(silent=True)

# Training the Regressor
regressor.fit(X_train, y_train)

# Predict the Values for X_test
y_pred = regressor.predict(X_test)

# Calculating R^2 Score
regressor_r2_score = r2_score(y_test, y_pred)

# Calculating Mean Squared Error
regressor_mse = mean_squared_error(y_test, y_pred)

# Calculating Root Mean Squared Error
regressor_rmse = np.sqrt(regressor_mse)

# Calculating Mean Absolute Error
regressor_mae = mean_absolute_error(y_test, y_pred)

In [ ]:
print( f"""
Regressor Name                    : {regressor.__class__.__name__}

Regressor R^2 Score               : {regressor_r2_score}

Regressor Mean Squared Error      : {regressor_mse}

Regressor Root Mean Squared Error : {regressor_rmse}

Regressor Mean Absolute Error     : {regressor_mae}
""")

In [ ]:
predictions = regressor.predict(test)
predictions

# Creating the Submission File

In [ ]:
# Using Sample Submission file to create the Submission file
sample_submission_data["num_sold"] = predictions

# Saving the Submission file
sample_submission_data.to_csv("submission.csv", index=False)